# Extended bullish engulfing - Analytics

### Import Library

In [103]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
import seaborn as sns

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [104]:
import os
from pathlib import Path
notebook_path = os.getcwd()
current_dir = Path(notebook_path)
csv_file = str(current_dir) + '/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [105]:
data = dataset.copy()

In [106]:
# data = data[data.index > '2020-11-01 00:00:00']

In [107]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765
...,...,...,...,...,...
2025-10-21 14:10:00,1927.5,1933.0,1918.7,1922.0,16658
2025-10-21 14:15:00,1922.0,1927.5,1919.2,1921.5,8361
2025-10-21 14:20:00,1921.5,1926.0,1902.9,1906.7,16009


In [108]:
def detect_extended_bullish_engulfing(df):
    cond1 = df['Close'].shift(2) < df['Open'].shift(2)  # nến 1 đỏ
    cond2 = df['Close'].shift(1) < df['Open'].shift(1)  # nến 2 đỏ
    cond3 = df['Close'].shift(1) < df['Close'].shift(2)
    cond4 = df['Close'] > df['Open']                    # nến 3 xanh
    cond5 = df['Close'] > df[['Open','Close']].shift(1).max(axis=1)
    cond6 = df['Close'] > df[['Open','Close']].shift(2).max(axis=1)
    df['bullish_engulfing'] = cond2 & cond3 & cond4 & cond5 & cond6
    return df

In [109]:
data = detect_extended_bullish_engulfing(data)

In [110]:
def detect_downtrend(df, price_col='Close', period=20):
    df = df.copy()
    df['SMA'] = df[price_col].rolling(period).mean()
    df['Downtrend'] = (df[price_col] < df['SMA']) & (df['SMA'].diff() < 0)
    return df

In [111]:
data = detect_downtrend(data)

In [112]:
def detect_pullback(df, price_col='Close', lookback=20, drop_pct=0.05):
    df['max_10'] = df["High"].rolling(10).max()
    df['min_3'] = df["Low"].rolling(3).min()
    df['DeepPullback'] = (df['max_10'] - df['Close'] > 2 * (df['Close'] - df['min_3']))
    return df

In [113]:
data = detect_pullback(data)

In [126]:
def cal_signal(df):
    cond1 = df['bullish_engulfing']
    # df["vol_avg"] = df["Volume"].rolling(20).mean()
    # cond2 = df["Volume"] > df["vol_avg"]
    cond2 = df["Volume"] < (df["Volume"].shift(1) + df["Volume"].shift(2))/2
    cond3 = df['Downtrend'].shift(1)
    cond4 = df['DeepPullback']
    df['long_signal'] = cond1 & cond2 & cond3 & cond4
    return df

In [127]:
data = cal_signal(data)

In [128]:
data[data.long_signal == True]

,Open,High,Low,Close,Volume,bullish_engulfing,SMA,Downtrend,max_10,min_3,DeepPullback,vol_avg,long_signal,SL,risk,reward
Date,,,,,,,,,,,,,,,,
2018-10-15 10:00:00,934.1,935.2,934.1,935.1,1605,True,938.085,False,939.3,933.6,True,2626.00,True,933.6,1.5,0.0
2018-11-13 09:20:00,869.0,870.5,869.0,869.9,2411,True,878.485,True,887.3,868.5,True,3470.40,True,868.5,1.4,0.0
2018-11-28 09:25:00,885.4,885.7,885.3,885.7,548,True,887.650,True,887.2,885.2,True,2748.95,True,885.2,0.5,0.0
2018-12-05 09:20:00,907.3,909.1,907.2,908.7,2583,True,911.855,False,914.8,906.7,True,3913.60,True,906.7,2.0,0.0
2018-12-06 09:15:00,912.7,913.0,912.6,913.0,1436,True,914.740,False,919.0,912.6,True,3685.75,True,912.6,0.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-18 09:10:00,1631.5,1633.3,1631.5,1632.7,2656,True,1638.115,True,1640.3,1630.8,True,5040.15,True,1630.8,1.9,0.0
2025-09-04 11:15:00,1845.6,1848.3,1844.9,1847.9,4817,True,1854.540,True,1859.0,1843.6,True,4326.90,True,1843.6,4.3,35.8
2025-09-19 10:15:00,1845.4,1846.8,1844.3,1846.0,2406,True,1850.365,True,1852.3,1843.8,True,5517.75,True,1843.8,2.2,0.0


## Calculate Risk & Reward

In [129]:
data['SL'] = data["min_3"]
data["risk"] = data['Close'] - data['SL']

In [130]:
%%time
data['reward'] = 0
for i, row in data.iterrows():
    if row['long_signal'] == True:
        current_date = row.name.strftime('%Y-%m-%d ').format()
        current_time = row.name
        entry_price = row['Close']
        stoploss = row['SL']
        reward = 0
        data_to_end_day = data[(data.index > current_time) & (data.index < current_date+' 14:30:00')]
        max_price = entry_price
        min_price = entry_price
        for k, wrow in data_to_end_day.iterrows():
            if wrow['High'] > max_price:
                max_price = wrow['High']
                reward = max_price - entry_price
            if wrow['Low'] <= stoploss:
                break
            # Long
        if reward: 
            data.at[i, 'reward'] = reward

CPU times: user 3.72 s, sys: 3.74 ms, total: 3.73 s
Wall time: 3.74 s


In [131]:
data[data.long_signal == True]

,Open,High,Low,Close,Volume,bullish_engulfing,SMA,Downtrend,max_10,min_3,DeepPullback,vol_avg,long_signal,SL,risk,reward
Date,,,,,,,,,,,,,,,,
2018-10-15 10:00:00,934.1,935.2,934.1,935.1,1605,True,938.085,False,939.3,933.6,True,2626.00,True,933.6,1.5,0.3
2018-11-13 09:20:00,869.0,870.5,869.0,869.9,2411,True,878.485,True,887.3,868.5,True,3470.40,True,868.5,1.4,8.1
2018-11-28 09:25:00,885.4,885.7,885.3,885.7,548,True,887.650,True,887.2,885.2,True,2748.95,True,885.2,0.5,2.7
2018-12-05 09:20:00,907.3,909.1,907.2,908.7,2583,True,911.855,False,914.8,906.7,True,3913.60,True,906.7,2.0,10.3
2018-12-06 09:15:00,912.7,913.0,912.6,913.0,1436,True,914.740,False,919.0,912.6,True,3685.75,True,912.6,0.4,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-07-18 09:10:00,1631.5,1633.3,1631.5,1632.7,2656,True,1638.115,True,1640.3,1630.8,True,5040.15,True,1630.8,1.9,2.7
2025-09-04 11:15:00,1845.6,1848.3,1844.9,1847.9,4817,True,1854.540,True,1859.0,1843.6,True,4326.90,True,1843.6,4.3,35.8
2025-09-19 10:15:00,1845.4,1846.8,1844.3,1846.0,2406,True,1850.365,True,1852.3,1843.8,True,5517.75,True,1843.8,2.2,1.3


In [132]:
total_risk = data[data.long_signal == True]["risk"].sum()
total_risk

np.float64(528.5000000000006)

In [133]:
total_reward = data[data.long_signal == True]["reward"].sum()
total_reward

np.float64(1158.3000000000015)

In [134]:
data[data.index == "2025-09-29 09:25:00"]

,Open,High,Low,Close,Volume,bullish_engulfing,SMA,Downtrend,max_10,min_3,DeepPullback,vol_avg,long_signal,SL,risk,reward
Date,,,,,,,,,,,,,,,,
2025-09-29 09:25:00,1838.4,1841.6,1837.8,1841.2,3942,True,1848.48,True,1862.2,1835.2,True,7072.65,True,1835.2,6.0,26.8
